# Image orientation

### Opening the dataset and creating the attribute array

In [14]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os 
import cv2

data_train = pd.read_csv('train.truth.csv')
imgs = data_train.fn
X_train = []
path = "C:/Users/Vinicius/Desktop/Deepersystems/train"

def create_X_train():
    for img in imgs:
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
        X_train.append([img_array])
            
create_X_train() 

###  Coding the output vector

In [21]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train = data_train.label
set(y_train)
y_train = label_encoder.fit_transform(y_train)

In [23]:
IMG_SIZE = 64
X_train = np.array(X_train).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X_train = X_train/255.0

### Testing CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time

dense_layers = [0, 1, 2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(256, (3, 3), input_shape=X_train.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(256, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))
                model.add(Dropout(0.2))
 

            model.add(Dense(1))
        model.add(Activation('sigmoid'))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                            metrics=['accuracy'],
                          )

            model.fit(X_train, y_train,
                      batch_size=64,
                      epochs=3,
                      validation_split=0.3, callbacks=[tensorboard])

### Testing NN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time

dense_layers = [0,1,2,3,4]
layer_sizes = [32,64,128,512]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        NAME = "{}-nodes-{}-dense-{}".format(layer_size, dense_layer, int(time.time()))
        tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
        print(NAME)
        
        model = tf.keras.models.Sequential()
        
        model.add(tf.keras.layers.Flatten())

        for _ in range(dense_layer):
            model.add(tf.keras.layers.Dense(layer_size, activation = tf.nn.relu))

        model.add(tf.keras.layers.Dense(4, activation = tf.nn.softmax))

        tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

        model.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

        model.fit(X_train,y_train,epochs=10, validation_split=0.3, callbacks=[tensorboard])

### The best setting

In [9]:
dense_layers = [2]
layer_sizes = [512]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        NAME = "{}-nodes-{}-dense-{}".format(layer_size, dense_layer, int(time.time()))
        tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
        print(NAME)
        
        model = tf.keras.models.Sequential()
        
        model.add(tf.keras.layers.Flatten())

        for _ in range(dense_layer):
            model.add(tf.keras.layers.Dense(layer_size, activation = tf.nn.relu))

        model.add(tf.keras.layers.Dense(4, activation = tf.nn.softmax))

        tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

        model.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

        model.fit(X_train,y_train,epochs=10, validation_split=0.3, callbacks=[tensorboard])

model.save('2x512-NN.model')

512-nodes-2-dense-1564493926
Train on 34227 samples, validate on 14669 samples
Epoch 1/10
34227/34227 [==============================] - 579s 17ms/sample - loss: 0.8011 - acc: 0.6862 - val_loss: 0.6449 - val_acc: 0.7511
Epoch 2/10
34227/34227 [==============================] - 358s 10ms/sample - loss: 0.6006 - acc: 0.7668 - val_loss: 0.5779 - val_acc: 0.7694
Epoch 3/10
34227/34227 [==============================] - 97s 3ms/sample - loss: 0.5393 - acc: 0.7941 - val_loss: 0.5698 - val_acc: 0.7774
Epoch 4/10
34227/34227 [==============================] - 86s 3ms/sample - loss: 0.5040 - acc: 0.8078 - val_loss: 0.5535 - val_acc: 0.7889
Epoch 5/10
34227/34227 [==============================] - 84s 2ms/sample - loss: 0.4772 - acc: 0.8176 - val_loss: 0.5119 - val_acc: 0.8043
Epoch 6/10
34227/34227 [==============================] - 105s 3ms/sample - loss: 0.4620 - acc: 0.8244 - val_loss: 0.6069 - val_acc: 0.7736
Epoch 7/10
34227/34227 [==============================] - 101s 3ms/sample - loss: 

In [27]:
import tensorflow as tf
model = tf.keras.models.load_model('2x512-NN.model')
path2 = "C:/Users/Vinicius/Desktop/Deepersystems/test"
IMG_SIZE = 64
data_test = []
predictions = []

def create_data_test():
    for img in os.listdir(path2):
        img_array = cv2.imread(os.path.join(path2,img), cv2.IMREAD_GRAYSCALE)
        new_array = np.array(img_array).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
        data_test.append([img, np.argmax(model.predict(new_array))])
            
create_data_test() 

Instructions for updating:
Colocations handled automatically by placer.


In [31]:
df = pd.DataFrame(np.array(data_test).reshape(5361,2), columns = ['fn', 'label'])

In [33]:
df.label = np.where(df.label=='0','rotated_left', df.label)
df.label = np.where(df.label=='1','rotated_right', df.label)
df.label = np.where(df.label=='2','upright', df.label)
df.label = np.where(df.label=='3','upside_down', df.label)

### Creating csv file

In [ ]:
df.to_csv(r'C:\Users\Vinicius\Desktop\Deepersystems\preds.csv')

### Rotating and saving the right images

In [90]:
X_test_correct = []
path3 = "C:/Users/Vinicius/Desktop/Deepersystems/corrected_images"

for index in df.index:
    img_array = cv2.imread(os.path.join(path2,df.fn[index]))
    name_jpg = df.fn[index]
    name_png = name_jpg[0:-3]+'PNG'
    if df.label[index] == 'rotated_left':
        new_img = rotateImage(cv2.imread(os.path.join(path2,df.fn[index])), 270)
        cv2.imwrite(os.path.join(path3,name_png),new_img)
        X_test_correct.append(new_img)
    if df.label[index] == 'rotated_right':
        new_img = rotateImage(cv2.imread(os.path.join(path2,df.fn[index])), 90)
        cv2.imwrite(os.path.join(path3,name_png),new_img)
        X_test_correct.append(new_img)
    if df.label[index] == 'upright':
        cv2.imwrite(os.path.join(path3,name_png),img_array)
        X_test_correct.append(img_array)
    if df.label[index] == 'upside_down':
        new_img = rotateImage(cv2.imread(os.path.join(path2,df.fn[index])), 180)
        cv2.imwrite(os.path.join(path3,name_png),new_img)
        X_test_correct.append(new_img)

### Image rotate function

In [10]:
def rotateImage(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

### Saving the numpy array (5361, 64, 64, 3)

In [93]:
import pickle
X_test_correct = np.array(X_test_correct)
pickle_out = open("X_test_correct.pickle", "wb")
pickle.dump(X_test_correct, pickle_out)
pickle_out.close()

In [92]:
from IPython.core.display import HTML
def css_styling():
    styles = open("styles/custom.css", "r").read()
    return HTML(styles)
css_styling()